In [42]:
import pandas as pd

In [43]:
# load the dataset
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/processed_ni_waste_data.csv')

# initial pre-processing
df.rename(columns={'AreaName': 'area'}, inplace=True) 
df.fillna(0, inplace=True)
df.replace('-', 0, inplace=True)

str_cols = ['quarter','quarter_name','fin_year','area']
for col in df.columns.tolist():
    if col in str_cols:
        pass
    else:
        df[col] = df[col].apply(lambda x: str(x).replace(',',''))
        df[col] = df[col].astype('float')

# sum up quarters to get annual totals instead of quarterly
df = df.groupby(['fin_year','area']).agg({
        'la_total_waste': 'sum',
        'la_reuse': 'sum',
        'la_dry_recycle': 'sum',
        'la_compost': 'sum',
        'la_energy_recovery_mixed_residual': 'sum',
        'la_energy_recovery_specific_stream': 'sum',
        'la_landfill': 'sum',
        'la_biodegradable_to_landfill': 'sum',
        'hh_total_waste': 'sum',
        'hh_reuse': 'sum',
        'hh_dry_recycle': 'sum',
        'hh_compost': 'sum',
        'hh_landfill': 'sum',
        'population': 'mean',
        'num_households': 'mean'
    }).reset_index()

# show sample of data
df.head()

,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,hh_total_waste,hh_reuse,hh_dry_recycle,hh_compost,hh_landfill,population,num_households
0,2006/07,Antrim,36791.0,0.0,8984.0,8352.0,0.0,0.0,19414.0,0.0,32732.0,0.0,7069.0,8352.0,17268.0,51059.0,0.0
1,2006/07,Ards,48673.0,0.0,5267.0,6661.0,0.0,0.0,36745.0,0.0,45592.0,0.0,4944.0,6661.0,33986.0,75767.0,0.0
2,2006/07,Armagh,29198.0,0.0,6330.0,3878.0,0.0,0.0,18989.0,0.0,25825.0,0.0,4599.0,3878.0,17346.0,56785.0,0.0
3,2006/07,Ballymena,38269.0,0.0,5520.0,4612.0,0.0,0.0,28137.0,0.0,35989.0,0.0,5520.0,4612.0,25856.0,61447.0,0.0
4,2006/07,Ballymoney,15923.0,0.0,2152.0,1341.0,0.0,0.0,12373.0,0.0,14115.0,0.0,2152.0,1341.0,10621.0,29392.0,0.0


In [44]:
# local authority (la) waste dataset
la_df = df[['fin_year','area','la_total_waste','la_reuse','la_dry_recycle','la_compost','la_energy_recovery_mixed_residual','la_energy_recovery_specific_stream','la_landfill','la_biodegradable_to_landfill','num_households','population']]
la_df.head()

,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,num_households,population
0,2006/07,Antrim,36791.0,0.0,8984.0,8352.0,0.0,0.0,19414.0,0.0,0.0,51059.0
1,2006/07,Ards,48673.0,0.0,5267.0,6661.0,0.0,0.0,36745.0,0.0,0.0,75767.0
2,2006/07,Armagh,29198.0,0.0,6330.0,3878.0,0.0,0.0,18989.0,0.0,0.0,56785.0
3,2006/07,Ballymena,38269.0,0.0,5520.0,4612.0,0.0,0.0,28137.0,0.0,0.0,61447.0
4,2006/07,Ballymoney,15923.0,0.0,2152.0,1341.0,0.0,0.0,12373.0,0.0,0.0,29392.0


In [45]:
# household (hh) waste dataset
hh_df = df[[
    'fin_year','area','hh_total_waste',
    'hh_reuse','hh_dry_recycle','hh_compost',
    'hh_landfill','num_households','population'
    ]]

# local authority (la) waste dataset
la_df = df[[
    'fin_year','area','la_total_waste'
    ,'la_reuse','la_dry_recycle','la_compost',
    'la_energy_recovery_mixed_residual',
    'la_energy_recovery_specific_stream','la_landfill',
    'la_biodegradable_to_landfill',
    'num_households','population'
    ]]

In [46]:
# PREPARE DATASETS

# prepare hh waste
hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100
hh_df['avg_landfill_waste_per_house'] = hh_df['hh_landfill'] / hh_df['num_households']
hh_df['hh_total_recycled'] = hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']

# prepare la waste
la_df['la_recycle_rate'] = (la_df['la_reuse'] + la_df['la_dry_recycle'] + la_df['la_compost']) / la_df['la_total_waste'] * 100
la_df['la_landfill_per_capita'] = la_df['la_landfill'] / la_df['population']
la_df['la_total_recycled'] = la_df['la_reuse'] + la_df['la_dry_recycle'] + la_df['la_compost']

/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m0000gn/T/ipykernel_65334/1826812704.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100
/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m0000gn/T/ipykernel_65334/1826812704.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh_df['avg_landfill_waste_per_house'] = hh_df['hh_landfill'] / hh_df['num_households']
/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m00

In [47]:
import json

# PREPARE JSON OF ALL SELECTABLE YEAR VALUES FOR FILTER OPTION
year_list = df['fin_year'].unique().tolist()
year_list.sort(reverse=True)
year_list = [str(year) for year in year_list]

output = []
for year in year_list:
    output.append({
        'label': year,
        'value': year
    })
# output_json = 


with open("output/selectable_fin_years.json", "w") as outfile: 
    json.dump(output, outfile)

In [48]:
# PREPARE OUTPUT FOR KPIs
import json

# KPI 1 - Current Period Recycle rate (%) (whole NI): 
output = {
    'kpi_1_output': {
        'values': {}
    },
    'kpi_2_output': {
        'values': {}
    },
    'kpi_3_output': {
        'values': {}
    },
    'kpi_4_output': {
        'values': {}
    },
    'kpi_5_output': {
        'min_values': {},
        'max_values': {}
    },
    'kpi_6_output': {
        'min_values': {},
        'max_values': {}
    },
    'kpi_7_output': {
        'min_values': {},
        'max_values': {}
    },
    'kpi_8_output': {
        'min_values': {},
        'max_values': {}
    }
}
for year in hh_df['fin_year'].unique():
    # KPI 1
    output['kpi_1_output']['values'][year] = '{0:.2f}'.format(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].mean())# hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].mean()

    # KPI 2
    output['kpi_2_output']['values'][year] = '{0:.2f}'.format(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].mean())

    # KPI 3
    output['kpi_3_output']['values'][year] = '{0:.2f}'.format(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].mean())

    # KPI 4
    output['kpi_4_output']['values'][year] = '{0:.2f}'.format(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill_per_capita'].mean())

    # KPI 5 - best + worst performance of KPI 1 above it
    idx_min = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].idxmin()
    output['kpi_5_output']['min_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].idxmax()
    output['kpi_5_output']['max_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_max]['area']

    # KPI 6 - best + worst performance of KPI 2 above it
    idx_min = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].idxmin()
    output['kpi_6_output']['min_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].idxmax()
    output['kpi_6_output']['max_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_max]['area']

    # KPI 7 - best + worst performance of KPI 3 above it
    idx_min = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].idxmin()
    output['kpi_7_output']['min_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].idxmax()
    output['kpi_7_output']['max_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_max]['area']

    # KPI 8 - best + worst performance of KPI 4 above it
    idx_min = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill_per_capita'].idxmin()
    output['kpi_8_output']['min_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill_per_capita'].idxmax()
    output['kpi_8_output']['max_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_max]['area']


with open("output/kpi_json_file.json", "w") as outfile: 
    json.dump(output, outfile)

In [49]:
# OLD - PREPARE OUTPUT FOR CHARTS
import json
output = {
    'chart_1': {},
    'chart_2': {},
    'chart_3': {},
    'chart_4': {},
    'chart_5': {},
    'chart_6': {
        'recycle_values': [],
        'landfill_values': [],
        'labels': [],
    },
}

for year in hh_df['fin_year'].unique():
    # CHART 1 - Recycle rate (%) by LA
    output['chart_1'][year] = {}
    output['chart_1'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].tolist()
    output['chart_1'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()

    # # CHART 2 - Landfill waste per household by LA
    # output['chart_2'][year] = {}
    # output['chart_2'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].tolist()
    # output['chart_2'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()
    
    # CHART 2 - Landfill waste by area by LA
    output['chart_2'][year] = {}
    output['chart_2'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].tolist()
    output['chart_2'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 2 - Landfill waste per household by LA

    # CHART 3 - Recycle rate (%) by HH
    output['chart_3'][year] = {}
    output['chart_3'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].tolist()
    output['chart_3'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # # CHART 4 - Landfill waste per household by HH
    # output['chart_4'][year] = {}
    # output['chart_4'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].tolist()
    # output['chart_4'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 4 - Landfill waste by area by HH
    output['chart_4'][year] = {}
    output['chart_4'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].tolist()
    output['chart_4'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 5 - Total Recycle vs Total Landfill by HH
    output['chart_5'][year] = {}
    output['chart_5'][year]['recycle_values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_total_recycled'].tolist()
    output['chart_5'][year]['landfill_values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].tolist()
    output['chart_5'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

     # CHART 6 - Total Recycle vs Total Landfill by HH
    output['chart_6']['recycle_values'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_total_recycled'].mean())
    output['chart_6']['landfill_values'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].mean())
    output['chart_6']['labels'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['fin_year'].unique().tolist()[0])


    # CHART 6 - Total Recycle vs Total Landfill over time

# with open("output/chart_json_file.json", "w") as outfile: 
#     json.dump(output, outfile)



In [51]:
# NEW - PREPARE OUTPUT FOR CHARTS
import json
output = {
    'chart_1': {},
    'chart_2': {},
    'chart_3': {},
    'chart_4': {},
    'chart_5': {},
    'trend_chart_1': {
        'recycle_values': [],
        'landfill_values': [],
        'labels': [],
    },
    'trend_chart_2': {
        'la_reuse': [],
        'la_dry_recycle': [],
        'la_compost': [],
        'la_landfill': [],
        'hh_reuse': [],
        'hh_dry_recycle': [],
        'hh_compost': [],
        'hh_landfill': [],
        'labels': [],
    },
}

for year in hh_df['fin_year'].unique():
    # CHART 1 - Recycle rate (%) LA vs HH
    output['chart_1'][year] = {'la': [], 'hh': [], 'labels': []}
    output['chart_1'][year]['la'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].tolist()
    output['chart_1'][year]['hh'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].tolist()
    output['chart_1'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 2 - Breakdown of waste by type LA vs HH
    output['chart_2'][year] = {'la': {'reuse': [], 'dry_recycle': [], 'compost': [], 'landfill': []}, 'hh': {'reuse': [], 'dry_recycle': [], 'compost': [], 'landfill': []}}
    output['chart_2'][year]['la']['reuse'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_reuse'].tolist()
    output['chart_2'][year]['la']['dry_recycle'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_dry_recycle'].tolist()
    output['chart_2'][year]['la']['compost'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_compost'].tolist()
    output['chart_2'][year]['la']['landfill'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].tolist()
    output['chart_2'][year]['hh']['reuse'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_reuse'].tolist()
    output['chart_2'][year]['hh']['dry_recycle'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_dry_recycle'].tolist()
    output['chart_2'][year]['hh']['compost'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_compost'].tolist()
    output['chart_2'][year]['hh']['landfill'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].tolist()
    output['chart_2'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 3 - Total waste (Tonnes) LA vs HH
    output['chart_3'][year] = {'la': [], 'hh': [], 'labels': []}
    output['chart_3'][year]['la'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_total_waste'].tolist()
    output['chart_3'][year]['hh'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_total_waste'].tolist()
    output['chart_3'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 4 - Landfill waste (Tonnes) LA vs HH
    output['chart_4'][year] = {'la': [], 'hh': [], 'labels': []}
    output['chart_4'][year]['la'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].tolist()
    output['chart_4'][year]['hh'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].tolist()
    output['chart_4'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 5 - Total Recycle vs Total Landfill by LA
    output['chart_5'][year] = {}
    output['chart_5'][year]['recycle_values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_total_recycled'].tolist()
    output['chart_5'][year]['landfill_values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].tolist()
    output['chart_5'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()
    
    # ------------------ #
    # CHARTS BEYOND THIS POINT ARE NOT CONNECTED TO DATE FILTER, SHOWS TREND OVER THE YEARS

    # CHART ? - Total Recycle vs Total Landfill by LA
    output['trend_chart_1']['recycle_values'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_total_recycled'].mean())
    output['trend_chart_1']['landfill_values'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].mean())
    output['trend_chart_1']['labels'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['fin_year'].unique().tolist()[0])

    # CHART ? - Breakdown of waste by type (Tonnes), Trend LA vs HH
    output['trend_chart_2']['la_reuse'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_reuse'].mean())
    output['trend_chart_2']['la_dry_recycle'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_dry_recycle'].mean())
    output['trend_chart_2']['la_compost'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_compost'].mean())
    output['trend_chart_2']['la_landfill'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].mean())
    output['trend_chart_2']['hh_reuse'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_reuse'].mean())
    output['trend_chart_2']['hh_landfill'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].mean())
    output['trend_chart_2']['hh_dry_recycle'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_dry_recycle'].mean())
    output['trend_chart_2']['hh_compost'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_compost'].mean())
    output['trend_chart_2']['labels'].append(la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['fin_year'].unique().tolist()[0])

    # output['chart_7']['recycle_values'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_total_recycled'].mean())
    # output['chart_7']['landfill_values'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].mean())
    # output['chart_7']['labels'].append(hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['fin_year'].unique().tolist()[0])



with open("output/chart_json_file.json", "w") as outfile: 
    json.dump(output, outfile)